# one-hot 작업이 번거로우니 이를 하지 않는 방법을 알아보자.

포인트
> model 설정할 때 loss에서 sparse_ 를 사용한다.

In [1]:
# 필요한 module 불러오기

import numpy as np
import pandas as pd
# 학습데이터와 테스트데이터를 분리해야 한다. (7:3 혹은 8:2로 분리)
from sklearn.model_selection import train_test_split
# 정규화
from sklearn.preprocessing import MinMaxScaler
# 우리 데이터에는 결측치와 이상치가 없다.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Raw Data Loading

df = pd.read_csv('./data/bmi.csv', skiprows=3)
display(df.head(), df.shape) # (20000, 3)

# 결측치, 이상치는 따로 없다.

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52


(20000, 3)

In [3]:
# 데이터 분리
# 학습데이터와 테스트데이터를 분리

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height', 'weight']].values,
                df['label'].values,
                test_size=0.3)
# 원래는 train_test_split() 안에는 2차원 ndarray가 들어와야 하는데, 알아서 데이터 추출해서 사용한다.
print(x_data_train, x_data_train.shape) # (14000, 2)
# 6000개는 테스트, 14000개는 트레이닝 용으로 분류

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_data_train)

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
print(x_data_train_norm, x_data_train_norm.shape) # (14000, 2)


[[196  66]
 [133  79]
 [133  52]
 ...
 [185  74]
 [124  78]
 [186  65]] (14000, 2)
[[0.95       0.68888889]
 [0.1625     0.97777778]
 [0.1625     0.37777778]
 ...
 [0.8125     0.86666667]
 [0.05       0.95555556]
 [0.825      0.66666667]] (14000, 2)


In [5]:
# Model
model = Sequential()

# Layer 추가
model.add(Flatten(input_shape=(2, )))
model.add(Dense(3, activation='softmax'))

# Model 설정
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
# loss에서의 sparse_
#     알아서 onehot으로 변경시켜서 동작한다.

# Model 학습
model.fit(x_data_train_norm,
         t_data_train,
         epochs=300,
         validation_split=0.2,
         verbose=1)

# 출력
# Epoch 1/300
# 350/350 [==============================] - 1s 1ms/step - loss: 1.0162 - accuracy: 0.4326 - val_loss: 1.0088 - val_accuracy: 0.4614
# Epoch 300/300
# 350/350 [==============================] - 0s 967us/step - loss: 0.3475 - accuracy: 0.9455 - val_loss: 0.3458 - val_accuracy: 0.9482
# <keras.callbacks.History at 0x2c7c00a79d0>

# loss, accuracy : 학습 데이터로 만든 결과물. 크게 중요치 않은 데이터.
# val_loss, val_accuracy : 중간중간 테스트데이터로 검증받은 확률. 이게 중요한 데이터다.
#     val_loss : loss니까 낮을 수록 좋은 결과
#     val_accuracy : 정확도. 높을 수록 좋다.

Epoch 1/300
350/350 [==============================] - 1s 1ms/step - loss: 1.0162 - accuracy: 0.4326 - val_loss: 1.0088 - val_accuracy: 0.4614
Epoch 2/300
350/350 [==============================] - 0s 859us/step - loss: 1.0013 - accuracy: 0.4923 - val_loss: 0.9947 - val_accuracy: 0.5114
Epoch 3/300
350/350 [==============================] - 0s 899us/step - loss: 0.9880 - accuracy: 0.5436 - val_loss: 0.9821 - val_accuracy: 0.5575
Epoch 4/300
350/350 [==============================] - 0s 918us/step - loss: 0.9760 - accuracy: 0.5831 - val_loss: 0.9704 - val_accuracy: 0.5943
Epoch 5/300
350/350 [==============================] - 0s 868us/step - loss: 0.9649 - accuracy: 0.6138 - val_loss: 0.9597 - val_accuracy: 0.6264
Epoch 6/300
350/350 [==============================] - 0s 863us/step - loss: 0.9546 - accuracy: 0.6406 - val_loss: 0.9496 - val_accuracy: 0.6504
Epoch 7/300
350/350 [==============================] - 0s 878us/step - loss: 0.9449 - accuracy: 0.6599 - val_loss: 0.9402 - val_accu

350/350 [==============================] - 0s 879us/step - loss: 0.6780 - accuracy: 0.8276 - val_loss: 0.6759 - val_accuracy: 0.8264
Epoch 58/300
350/350 [==============================] - 0s 851us/step - loss: 0.6744 - accuracy: 0.8286 - val_loss: 0.6723 - val_accuracy: 0.8279
Epoch 59/300
350/350 [==============================] - 0s 911us/step - loss: 0.6709 - accuracy: 0.8286 - val_loss: 0.6689 - val_accuracy: 0.8289
Epoch 60/300
350/350 [==============================] - 0s 867us/step - loss: 0.6675 - accuracy: 0.8317 - val_loss: 0.6654 - val_accuracy: 0.8289
Epoch 61/300
350/350 [==============================] - 0s 909us/step - loss: 0.6641 - accuracy: 0.8306 - val_loss: 0.6620 - val_accuracy: 0.8293
Epoch 62/300
350/350 [==============================] - 0s 866us/step - loss: 0.6607 - accuracy: 0.8340 - val_loss: 0.6587 - val_accuracy: 0.8318
Epoch 63/300
350/350 [==============================] - 0s 853us/step - loss: 0.6574 - accuracy: 0.8372 - val_loss: 0.6554 - val_accuracy

350/350 [==============================] - 0s 1ms/step - loss: 0.5337 - accuracy: 0.8813 - val_loss: 0.5324 - val_accuracy: 0.8789
Epoch 114/300
350/350 [==============================] - 0s 1ms/step - loss: 0.5318 - accuracy: 0.8811 - val_loss: 0.5305 - val_accuracy: 0.8789
Epoch 115/300
350/350 [==============================] - 0s 1ms/step - loss: 0.5300 - accuracy: 0.8816 - val_loss: 0.5287 - val_accuracy: 0.8789
Epoch 116/300
350/350 [==============================] - 0s 942us/step - loss: 0.5282 - accuracy: 0.8815 - val_loss: 0.5269 - val_accuracy: 0.8789
Epoch 117/300
350/350 [==============================] - 0s 940us/step - loss: 0.5264 - accuracy: 0.8817 - val_loss: 0.5250 - val_accuracy: 0.8800
Epoch 118/300
350/350 [==============================] - 0s 923us/step - loss: 0.5245 - accuracy: 0.8814 - val_loss: 0.5232 - val_accuracy: 0.8786
Epoch 119/300
350/350 [==============================] - 0s 909us/step - loss: 0.5227 - accuracy: 0.8814 - val_loss: 0.5214 - val_accuracy

Epoch 169/300
350/350 [==============================] - 0s 1ms/step - loss: 0.4515 - accuracy: 0.9063 - val_loss: 0.4502 - val_accuracy: 0.9039
Epoch 170/300
350/350 [==============================] - 0s 953us/step - loss: 0.4503 - accuracy: 0.9066 - val_loss: 0.4491 - val_accuracy: 0.9043
Epoch 171/300
350/350 [==============================] - 0s 901us/step - loss: 0.4492 - accuracy: 0.9067 - val_loss: 0.4479 - val_accuracy: 0.9046
Epoch 172/300
350/350 [==============================] - 0s 873us/step - loss: 0.4480 - accuracy: 0.9072 - val_loss: 0.4468 - val_accuracy: 0.9046
Epoch 173/300
350/350 [==============================] - 0s 918us/step - loss: 0.4469 - accuracy: 0.9072 - val_loss: 0.4457 - val_accuracy: 0.9043
Epoch 174/300
350/350 [==============================] - 0s 952us/step - loss: 0.4458 - accuracy: 0.9077 - val_loss: 0.4445 - val_accuracy: 0.9046
Epoch 175/300
350/350 [==============================] - 0s 1ms/step - loss: 0.4446 - accuracy: 0.9078 - val_loss: 0.443

Epoch 225/300
350/350 [==============================] - 0s 965us/step - loss: 0.3972 - accuracy: 0.9225 - val_loss: 0.3958 - val_accuracy: 0.9232
Epoch 226/300
350/350 [==============================] - 0s 969us/step - loss: 0.3964 - accuracy: 0.9244 - val_loss: 0.3950 - val_accuracy: 0.9243
Epoch 227/300
350/350 [==============================] - 0s 942us/step - loss: 0.3956 - accuracy: 0.9246 - val_loss: 0.3942 - val_accuracy: 0.9293
Epoch 228/300
350/350 [==============================] - 0s 950us/step - loss: 0.3948 - accuracy: 0.9287 - val_loss: 0.3934 - val_accuracy: 0.9311
Epoch 229/300
350/350 [==============================] - 0s 951us/step - loss: 0.3940 - accuracy: 0.9305 - val_loss: 0.3926 - val_accuracy: 0.9304
Epoch 230/300
350/350 [==============================] - 0s 1ms/step - loss: 0.3933 - accuracy: 0.9298 - val_loss: 0.3918 - val_accuracy: 0.9307
Epoch 231/300
350/350 [==============================] - 0s 980us/step - loss: 0.3925 - accuracy: 0.9310 - val_loss: 0.3

Epoch 281/300
350/350 [==============================] - 0s 926us/step - loss: 0.3584 - accuracy: 0.9418 - val_loss: 0.3568 - val_accuracy: 0.9425
Epoch 282/300
350/350 [==============================] - 0s 1ms/step - loss: 0.3578 - accuracy: 0.9421 - val_loss: 0.3562 - val_accuracy: 0.9429
Epoch 283/300
350/350 [==============================] - 0s 975us/step - loss: 0.3572 - accuracy: 0.9424 - val_loss: 0.3556 - val_accuracy: 0.9432
Epoch 284/300
350/350 [==============================] - 0s 950us/step - loss: 0.3566 - accuracy: 0.9425 - val_loss: 0.3550 - val_accuracy: 0.9432
Epoch 285/300
350/350 [==============================] - 0s 974us/step - loss: 0.3561 - accuracy: 0.9426 - val_loss: 0.3544 - val_accuracy: 0.9439
Epoch 286/300
350/350 [==============================] - 0s 946us/step - loss: 0.3555 - accuracy: 0.9427 - val_loss: 0.3538 - val_accuracy: 0.9439
Epoch 287/300
350/350 [==============================] - 0s 1ms/step - loss: 0.3549 - accuracy: 0.9428 - val_loss: 0.353

In [16]:
# 모델의 evaluation(평가) 진행
model.evaluate(x_data_test_norm, t_data_test_onehot)
# 출력
#     188/188 [==============================] - 0s 633us/step - loss: 0.2742 - accuracy: 0.9707
#     [0.2741805911064148, 0.9706666469573975]
#     [최종 loss값, 최종 accuracy값]


188/188 [==============================] - 0s 707us/step - loss: 0.2742 - accuracy: 0.9707


[0.2741805911064148, 0.9706666469573975]

In [20]:
# 예측
predict_data = np.array([[180, 76]])
scaled_predict_data = scaler.transform(predict_data)
print(model.predict(scaled_predict_data))


1/1 [==============================] - 0s 16ms/step
[[0.02958116 0.6650098  0.30540907]]
